In [1]:
import time

import numba
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

import adaptoctree.tree as tree
import adaptoctree.morton as morton

In [26]:
# Generate randomly distributed 3D points in a unit cube domain
n_points = [int(i) for i in [1e4, 1e5, 6e6]]

pointsvec = [np.random.rand(n, 3) for n in n_points]
densitiesvec = [np.random.rand(n) for n in n_points]
# Build parameters
max_level = 16 # Maximum level allowed in AdaptOctree
max_points = 150 # The maximum points per node constraint
start_level = 1 # Initial level, to start tree construction from

In [3]:
nruns = 5

buildtimes = [[] for _ in range(nruns)]
balancetimes = [[] for _ in range(nruns)]

for j in range(len(n_points)):
    for i in range(nruns):
        s = time.time()
        unbalanced = tree.build(pointsvec[j], max_level, max_points, start_level)

        buildtimes[i].append(time.time()-s)

        # Now balance the unbalanced tree
        depth = np.max(morton.find_level(unbalanced)) # Maximum depth achieved in octree
#         print(n_points[j])
#         print(depth)
        s = time.time()
        balanced = tree.balance(unbalanced, depth)
#         print(balanced.shape)
#         print()
        balancetimes[i].append(time.time()-s)

buildtimes = np.array(buildtimes)
balancetimes = np.array(balancetimes)

In [4]:
(buildtimes.mean(axis=0) + balancetimes.mean(axis=0))

array([0.01593881, 0.21964183, 2.8167562 ])

In [5]:
(buildtimes.std(axis=0) + balancetimes.std(axis=0))

array([0.00027922, 0.00529059, 0.02003295])

In [27]:
# create a list of source instances
nruns = 5

exbuildtimes = [[] for _ in range(len(n_points))]

for i in range(len(n_points)):
    if n_points[i] == 6e6:
        for j in range(nruns):
            sources = laplace.init_sources(pointsvec[i], densitiesvec[i])
            targets = laplace.init_targets(pointsvec[i])

            p = 5

            fmm = laplace.LaplaceFmm(
                p=p, 
                ncrit=150, 
                filename=f'C{p}E{p}.dat'
            )    
            s = time.time()
            tree = laplace.setup(sources, targets, fmm)
            exbuildtimes[i].append(time.time()-s)
        
exbuildtimes = np.array(exbuildtimes)

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

In [32]:

len(tree.leafs)

260639

In [29]:
np.array(exbuildtimes[2]).mean()

3.9665105819702147

In [20]:
# exbuildtimes = exbuildtimes[:,1:]
exbuildtimes.mean(axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [17]:
exbuildtimes.mean(axis=1)

array([0.06988039, 0.0951509 , 3.94565573])

In [18]:
exbuildtimes.std(axis=1)

array([0.12301845, 0.0899455 , 0.15130458])